# Order Logic Test - UNH Bull Put Spread

This notebook tests the exact same order placement logic from `vertical_spread_order.py` using the UNH Bull Put Spread that generated Error 321 on July 31st, 2025.

**Test Case Details:**
- Symbol: UNH
- Strategy: Bull Put Spread
- Expiry: 2025-08-22
- Strike Sell: 240P (higher strike)
- Strike Buy: 230P (lower strike)
- Expected Credit: ~$1.95

In [12]:
# Import all the same modules as vertical_spread_order.py
import sqlite3
import pandas as pd
import datetime
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract, ComboLeg
from ibapi.order import Order
from ibapi.utils import iswrapper
import time
import threading
import logging
import os
import random
import json

# Set up logging for the test
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [13]:
# EXACT COPY of IBWrapper class from vertical_spread_order.py
class IBWrapper(EWrapper):
    def __init__(self):
        super().__init__()
        self.next_order_id = None
        self.contract_details = {}
        self.mid_prices = {}
        self.combo_ids = {}
        self.market_status = "unknown"  # To track market status

    @iswrapper
    def nextValidId(self, orderId: int):
        self.next_order_id = orderId
        logger.info(f"Next Valid Order ID: {orderId}")
    
    @iswrapper
    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType in (1, 2):  # Bid or Ask
            if reqId not in self.mid_prices:
                self.mid_prices[reqId] = {"bid": None, "ask": None, "last": None, "model": None}
            self.mid_prices[reqId]["bid" if tickType == 1 else "ask"] = price
            logger.info(f"Received {'bid' if tickType == 1 else 'ask'} price for req_id {reqId}: {price}")
    
    @iswrapper
    def tickOptionComputation(self, reqId, tickType, tickAttrib, impliedVol, delta, optPrice, pvDividend, gamma, vega, theta, undPrice):
        if optPrice is not None and tickType in (12, 13):  # 12 = last price, 13 = model price
            if reqId not in self.mid_prices:
                self.mid_prices[reqId] = {"bid": None, "ask": None, "last": None, "model": None}
            self.mid_prices[reqId]["last" if tickType == 12 else "model"] = optPrice
            logger.info(f"Received {'last' if tickType == 12 else 'model'} price for req_id {reqId}: {optPrice}")

    @iswrapper
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", connectionClosed=False):
        logger.error(f"Error {reqId}: {errorCode} - {errorString}")
        # Check for market closed error codes
        if errorCode in [2104, 2119]:  # Known IBKR error codes for market closed
            self.market_status = "closed"
            logger.warning(f"Market appears to be closed: {errorString}")
        
    @iswrapper
    def contractDetails(self, reqId, contractDetails):
        if reqId in self.combo_ids:
            self.combo_ids[reqId] = {
                "conId": contractDetails.contract.conId,
                "symbol": contractDetails.contract.symbol,
                "strike": contractDetails.contract.strike,
                "right": contractDetails.contract.right,
                "expiry": contractDetails.contract.lastTradeDateOrContractMonth
            }
            logger.info(f"Received contract details: {self.combo_ids[reqId]['symbol']} {self.combo_ids[reqId]['expiry']} {self.combo_ids[reqId]['strike']} {self.combo_ids[reqId]['right']}, conId: {self.combo_ids[reqId]['conId']}")

    @iswrapper
    def contractDetailsEnd(self, reqId):
        logger.info(f"Contract details request {reqId} completed")

    @iswrapper
    def openOrder(self, orderId, contract, order, orderState):
        logger.info(f"Order {orderId} status: {orderState.status}")

    @iswrapper
    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        logger.info(f"Order {orderId} status update: {status}, filled: {filled}, remaining: {remaining}")

In [14]:
# EXACT COPY of IBClient and IBApp classes from vertical_spread_order.py
class IBClient(EClient):
    def __init__(self, wrapper):
        super().__init__(wrapper)

class IBApp(IBWrapper, IBClient):
    def __init__(self):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)

    def create_option_contract(self, symbol, expiry, strike, right):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = expiry.replace("-", "")
        contract.strike = strike
        contract.right = right
        contract.multiplier = "100"
        
        logger.info(f"Created option contract: {symbol} {contract.lastTradeDateOrContractMonth} {strike} {right}")
        return contract
    
    def create_combo_contract(self, symbol, leg_contracts, contract_ids):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "BAG"
        contract.exchange = "SMART"
        contract.currency = "USD"
        
        buy_contract, sell_contract = leg_contracts
        buy_conId, sell_conId = contract_ids
        
        # Create legs
        leg1 = ComboLeg()
        leg1.conId = buy_conId
        leg1.ratio = 1
        leg1.action = "BUY"
        leg1.exchange = "SMART"
        
        leg2 = ComboLeg()
        leg2.conId = sell_conId
        leg2.ratio = 1
        leg2.action = "SELL"
        leg2.exchange = "SMART"
        
        contract.comboLegs = [leg1, leg2]
        logger.info(f"Created combo contract for {symbol} with 2 legs")
        return contract
    
    def create_limit_order(self, action, quantity, price):
        """
        Create a limit order with time in force of 1 DAY
        
        Parameters:
        action (str): "BUY" or "SELL"
        quantity (int): Number of contracts
        price (float): Desired price
        
        Returns:
        Order: IBKR order object
        """
        # Ensure price is negative for vertical spreads
        # Negative price means we're collecting a credit
        if price > 0:
            price = -abs(price)
        
        # Round price to nearest $0.05 increment for most options
        price_increment = 0.05
        rounded_price = round(price / price_increment) * price_increment
        
        # Ensure at least 2 decimal places
        rounded_price = round(rounded_price, 2)
        
        order = Order()
        order.action = action
        order.orderType = "LMT"
        order.totalQuantity = quantity  # Always set to 1
        order.lmtPrice = rounded_price
        order.tif = "DAY"  # Set time in force to 1 DAY
        order.transmit = True
        
        logger.info(f"Created {action} limit order: Quantity={quantity}, Price=${rounded_price:.2f}, TIF=DAY")
        return order
    
    def get_contract_details(self, contract, req_id):
        self.combo_ids[req_id] = None
        self.reqContractDetails(req_id, contract)
        
        wait_time = 8
        start_time = time.time()
        while self.combo_ids[req_id] is None and time.time() - start_time < wait_time:
            time.sleep(0.1)
        return self.combo_ids.get(req_id)

    def get_price_data(self, contract, req_id):
        """
        Request and retrieve market data for a contract.
        
        Parameters:
        contract (Contract): Option contract
        req_id (int): Request ID to track this specific request
        
        Returns:
        dict: Pricing data including bid, ask, last, and model prices
        """
        logger.info(f"Requesting market data for {contract.symbol} {contract.strike} {contract.right} (req_id: {req_id})")
        
        # Clear any previous data for this req_id
        if req_id in self.mid_prices:
            del self.mid_prices[req_id]
            
        # Initialize with empty data
        self.mid_prices[req_id] = {"bid": None, "ask": None, "last": None, "model": None}
        
        # Request market data
        self.reqMktData(req_id, contract, "", False, False, [])
        
        # Wait for data to arrive
        wait_time = 8  # Wait up to 5 seconds for market data
        start_time = time.time()
        while time.time() - start_time < wait_time:
            # Check if we have received both bid and ask
            if (self.mid_prices[req_id]["bid"] is not None and 
                self.mid_prices[req_id]["ask"] is not None):
                # We have our data, no need to wait further
                break
            time.sleep(0.2)
        
        # Cancel the market data subscription
        self.cancelMktData(req_id)
        
        # Return the price data
        data = self.mid_prices.get(req_id, {"bid": None, "ask": None, "last": None, "model": None})
        
        if data["bid"] is not None and data["ask"] is not None:
            logger.info(f"Received market data for req_id {req_id}: Bid={data['bid']}, Ask={data['ask']}")
        else:
            logger.warning(f"Incomplete market data for req_id {req_id}: Bid={data['bid']}, Ask={data['ask']}")
            
        return data

In [15]:
# EXACT COPY of calculate_spread_premium function from vertical_spread_order.py
def calculate_spread_premium(sell_data, buy_data):
    """
    Calculate the current market premium for a vertical spread
    
    Parameters:
    sell_data (dict): Market data for the sell leg
    buy_data (dict): Market data for the buy leg
    
    Returns:
    tuple: (premium amount, description of calculation method used)
    """
    premium = None
    calc_method = "unknown"
    
    # Check for -1.0 prices which indicate no market data available
    if (sell_data["bid"] == -1.0 or sell_data["ask"] == -1.0 or 
        buy_data["bid"] == -1.0 or buy_data["ask"] == -1.0):
        return None, "no_market_data"
    
    # Check if we have valid bid/ask for both legs
    if (sell_data["bid"] is not None and sell_data["ask"] is not None and
        buy_data["bid"] is not None and buy_data["ask"] is not None and
        sell_data["bid"] > 0 and sell_data["ask"] > 0 and
        buy_data["bid"] > 0 and buy_data["ask"] > 0):
        # Use midpoint prices for more accurate premium calculation
        sell_midpoint = (sell_data["bid"] + sell_data["ask"]) / 2
        buy_midpoint = (buy_data["bid"] + buy_data["ask"]) / 2
        premium = (sell_midpoint - buy_midpoint) * 100  # Convert to premium per contract
        calc_method = "midpoint"
    # Fallback to bid/ask if midpoints can't be calculated
    elif (sell_data["bid"] is not None and buy_data["ask"] is not None and
          sell_data["bid"] > 0 and buy_data["ask"] > 0):
        # Conservative estimate: what we can sell at bid and buy at ask
        premium = (sell_data["bid"] - buy_data["ask"]) * 100
        calc_method = "conservative"
    # Use last prices if available
    elif (sell_data["last"] is not None and buy_data["last"] is not None and
          sell_data["last"] > 0 and buy_data["last"] > 0):
        premium = (sell_data["last"] - buy_data["last"]) * 100
        calc_method = "last"
    # Use model prices if available
    elif (sell_data["model"] is not None and buy_data["model"] is not None and
          sell_data["model"] > 0 and buy_data["model"] > 0):
        premium = (sell_data["model"] - buy_data["model"]) * 100
        calc_method = "model"
        
    return premium, calc_method

In [16]:
# Test configuration - UNH Bull Put Spread from the error log
TEST_CONFIG = {
    'ticker': 'UNH',
    'expiry': '2025-08-22',
    'strategy_type': 'Bull Put',
    'strike_buy': 230.0,   # Lower strike (buy)
    'strike_sell': 240.0,  # Higher strike (sell)
    'estimated_premium': 197.0,  # From the log: $197.00
    'ibkr_host': '127.0.0.1',
    'ibkr_port': 4002,  # IB Gateway paper trading port
    'client_id': 5,  # Same as in the error log
    'dry_run': False  # Set to False to actually place orders
}

print("Test Configuration:")
for key, value in TEST_CONFIG.items():
    print(f"  {key}: {value}")

Test Configuration:
  ticker: UNH
  expiry: 2025-08-22
  strategy_type: Bull Put
  strike_buy: 230.0
  strike_sell: 240.0
  estimated_premium: 197.0
  ibkr_host: 127.0.0.1
  ibkr_port: 4002
  client_id: 5
  dry_run: False


In [17]:
# Test the contract creation logic - EXACT COPY from vertical_spread_order.py lines 411-415
def test_contract_creation(config):
    """Test contract creation using exact logic from vertical_spread_order.py"""
    
    print("\n=== Testing Contract Creation ===")
    
    # Create IBApp instance (but don't connect yet)
    app = IBApp()
    
    ticker = config['ticker']
    expiry = config['expiry']
    strategy_type = config['strategy_type']
    strike_buy = config['strike_buy']
    strike_sell = config['strike_sell']
    
    print(f"Processing {strategy_type} for {ticker}, expiry {expiry}")
    
    # EXACT COPY of logic from vertical_spread_order.py lines 411-415
    if strategy_type == 'Bull Put':
        sell_contract = app.create_option_contract(ticker, expiry, strike_sell, "P")
        buy_contract = app.create_option_contract(ticker, expiry, strike_buy, "P")
        combo_action = "BUY"
        option_right = "P"
    else:
        print(f"Strategy type {strategy_type} not implemented in test")
        return None, None, None, None
    
    print(f"\nCreated contracts:")
    print(f"  Sell contract: {sell_contract.symbol} {sell_contract.lastTradeDateOrContractMonth} {sell_contract.strike} {sell_contract.right}")
    print(f"  Buy contract: {buy_contract.symbol} {buy_contract.lastTradeDateOrContractMonth} {buy_contract.strike} {buy_contract.right}")
    print(f"  Combo action: {combo_action}")
    print(f"  Option right: {option_right}")
    
    return sell_contract, buy_contract, combo_action, option_right

# Run the test
sell_contract, buy_contract, combo_action, option_right = test_contract_creation(TEST_CONFIG)

2025-08-01 05:54:58,706 - INFO - Created option contract: UNH 20250822 240.0 P
2025-08-01 05:54:58,707 - INFO - Created option contract: UNH 20250822 230.0 P



=== Testing Contract Creation ===
Processing Bull Put for UNH, expiry 2025-08-22

Created contracts:
  Sell contract: UNH 20250822 240.0 P
  Buy contract: UNH 20250822 230.0 P
  Combo action: BUY
  Option right: P


In [18]:
# Test the combo contract creation - EXACT COPY from vertical_spread_order.py lines 499-500
def test_combo_contract_creation(sell_contract, buy_contract):
    """Test combo contract creation using exact logic from vertical_spread_order.py"""
    
    print("\n=== Testing Combo Contract Creation ===")
    
    # Create IBApp instance (but don't connect yet)
    app = IBApp()
    
    # Mock contract IDs (as they would come from IB)
    # From the log: conId: 797608409 (240P), conId: 797608326 (230P)
    sell_conId = 797608409  # 240P strike
    buy_conId = 797608326   # 230P strike
    
    print(f"Using mock contract IDs:")
    print(f"  Sell contract ({sell_contract.strike}P) conId: {sell_conId}")
    print(f"  Buy contract ({buy_contract.strike}P) conId: {buy_conId}")
    
    # EXACT COPY of logic from vertical_spread_order.py line 499-500
    combo_contract = app.create_combo_contract(
        TEST_CONFIG['ticker'], [buy_contract, sell_contract], [buy_conId, sell_conId])
    
    print(f"\nCombo contract details:")
    print(f"  Symbol: {combo_contract.symbol}")
    print(f"  SecType: {combo_contract.secType}")
    print(f"  Exchange: {combo_contract.exchange}")
    print(f"  Currency: {combo_contract.currency}")
    print(f"  Number of legs: {len(combo_contract.comboLegs)}")
    
    for i, leg in enumerate(combo_contract.comboLegs):
        print(f"  Leg {i+1}: conId={leg.conId}, ratio={leg.ratio}, action={leg.action}, exchange={leg.exchange}")
    
    return combo_contract, sell_conId, buy_conId

# Run the test
if sell_contract and buy_contract:
    combo_contract, sell_conId, buy_conId = test_combo_contract_creation(sell_contract, buy_contract)
else:
    print("Skipping combo contract test - contracts not created")

2025-08-01 05:54:58,746 - INFO - Created combo contract for UNH with 2 legs



=== Testing Combo Contract Creation ===
Using mock contract IDs:
  Sell contract (240.0P) conId: 797608409
  Buy contract (230.0P) conId: 797608326

Combo contract details:
  Symbol: UNH
  SecType: BAG
  Exchange: SMART
  Currency: USD
  Number of legs: 2
  Leg 1: conId=797608326, ratio=1, action=BUY, exchange=SMART
  Leg 2: conId=797608409, ratio=1, action=SELL, exchange=SMART


In [19]:
# Test the order creation logic - EXACT COPY from vertical_spread_order.py
def test_order_creation(combo_action, estimated_premium):
    """Test order creation using exact logic from vertical_spread_order.py"""
    
    print("\n=== Testing Order Creation ===")
    
    # Create IBApp instance (but don't connect yet)
    app = IBApp()
    
    # EXACT COPY of logic from vertical_spread_order.py lines 456-471
    
    # Mock market data (using values from the log)
    # From log: 240P Bid=3.6, Ask=3.8 | 230P Bid=1.62, Ask=1.84
    sell_price_data = {"bid": 3.6, "ask": 3.8, "last": None, "model": None}
    buy_price_data = {"bid": 1.62, "ask": 1.84, "last": None, "model": None}
    
    print(f"Mock market data:")
    print(f"  Sell leg (240P): Bid={sell_price_data['bid']}, Ask={sell_price_data['ask']}")
    print(f"  Buy leg (230P): Bid={buy_price_data['bid']}, Ask={buy_price_data['ask']}")
    
    # Calculate the actual market premium
    market_premium, calc_method = calculate_spread_premium(sell_price_data, buy_price_data)
    
    print(f"\nPremium calculation:")
    print(f"  Market premium: ${market_premium:.2f} using {calc_method} method")
    print(f"  Estimated premium: ${estimated_premium:.2f}")
    
    # Determine if we should place an order
    place_order = False
    
    # Ensure price is negative for credit collected
    limit_price = -abs(float(estimated_premium) / 100)
    limit_price = round(limit_price * 20) / 20  # Round to nearest 0.05 increment
    
    if market_premium is not None:
        print(f"\nOrder decision logic:")
        # Compare estimated vs market premium
        if market_premium >= float(estimated_premium):
            place_order = True
            print(f"  Market premium (${market_premium:.2f}) is >= estimated premium (${estimated_premium:.2f}) -> PLACE ORDER")
            # Use market premium for limit price when it's better
            market_limit_price = -abs(market_premium / 100)  # Make sure it's negative
            market_limit_price = round(market_limit_price * 20) / 20  # Round to nearest 0.05
            limit_price = market_limit_price
        else:
            print(f"  Market premium (${market_premium:.2f}) is less than estimated premium (${estimated_premium:.2f}) -> NO ORDER")
    
    print(f"\nFinal order parameters:")
    print(f"  Place order: {place_order}")
    print(f"  Limit price: ${limit_price:.2f} per share")
    print(f"  Action: {combo_action}")
    print(f"  Quantity: 1")
    print(f"  TIF: DAY")
    
    if place_order:
        # Create the order using exact logic
        order = app.create_limit_order(combo_action, 1, limit_price)
        
        print(f"\nCreated order object:")
        print(f"  Action: {order.action}")
        print(f"  Order type: {order.orderType}")
        print(f"  Total quantity: {order.totalQuantity}")
        print(f"  Limit price: {order.lmtPrice}")
        print(f"  Time in force: {order.tif}")
        print(f"  Transmit: {order.transmit}")
        
        return order, place_order, limit_price
    
    return None, place_order, limit_price

# Run the test
if combo_action:
    order, should_place, final_price = test_order_creation(combo_action, TEST_CONFIG['estimated_premium'])
else:
    print("Skipping order creation test - combo action not defined")

2025-08-01 05:54:58,772 - INFO - Created BUY limit order: Quantity=1, Price=$-1.95, TIF=DAY



=== Testing Order Creation ===
Mock market data:
  Sell leg (240P): Bid=3.6, Ask=3.8
  Buy leg (230P): Bid=1.62, Ask=1.84

Premium calculation:
  Market premium: $197.00 using midpoint method
  Estimated premium: $197.00

Order decision logic:
  Market premium ($197.00) is >= estimated premium ($197.00) -> PLACE ORDER

Final order parameters:
  Place order: True
  Limit price: $-1.95 per share
  Action: BUY
  Quantity: 1
  TIF: DAY

Created order object:
  Action: BUY
  Order type: LMT
  Total quantity: 1
  Limit price: -1.95
  Time in force: DAY
  Transmit: True


In [20]:
# Test the complete order flow (DRY RUN - no actual connection)
def test_complete_order_flow(config, dry_run=True):
    """Test the complete order flow without connecting to IB"""
    
    print("\n=== Testing Complete Order Flow ===")
    print(f"DRY RUN MODE: {dry_run}")
    
    # Simulate the exact sequence from vertical_spread_order.py
    
    print("\n1. Contract Creation:")
    app = IBApp()
    
    # Create contracts (Bull Put strategy)
    sell_contract = app.create_option_contract(config['ticker'], config['expiry'], config['strike_sell'], "P")
    buy_contract = app.create_option_contract(config['ticker'], config['expiry'], config['strike_buy'], "P")
    combo_action = "BUY"
    
    print("\n2. Mock Contract Details (from IB):")
    # Mock the contract details that would come from IB
    sell_details = {'conId': 797608409, 'symbol': 'UNH', 'strike': 240.0, 'right': 'P', 'expiry': '20250822'}
    buy_details = {'conId': 797608326, 'symbol': 'UNH', 'strike': 230.0, 'right': 'P', 'expiry': '20250822'}
    
    print(f"  Sell details: {sell_details}")
    print(f"  Buy details: {buy_details}")
    
    print("\n3. Mock Market Data (from IB):")
    # Mock market data from the log
    sell_price_data = {"bid": 3.6, "ask": 3.8, "last": None, "model": None}
    buy_price_data = {"bid": 1.62, "ask": 1.84, "last": None, "model": None}
    
    print(f"  240P: Bid={sell_price_data['bid']}, Ask={sell_price_data['ask']}")
    print(f"  230P: Bid={buy_price_data['bid']}, Ask={buy_price_data['ask']}")
    
    print("\n4. Premium Calculation:")
    market_premium, calc_method = calculate_spread_premium(sell_price_data, buy_price_data)
    print(f"  Calculated market premium: ${market_premium:.2f} using {calc_method} method")
    print(f"  Estimated premium: ${config['estimated_premium']:.2f}")
    
    print("\n5. Order Decision:")
    place_order = market_premium >= float(config['estimated_premium'])
    print(f"  Should place order: {place_order}")
    
    if place_order:
        print("\n6. Combo Contract Creation:")
        combo_contract = app.create_combo_contract(
            config['ticker'], [buy_contract, sell_contract], [buy_details['conId'], sell_details['conId']])
        
        print("\n7. Order Creation:")
        market_limit_price = -abs(market_premium / 100)
        market_limit_price = round(market_limit_price * 20) / 20
        order = app.create_limit_order(combo_action, 1, market_limit_price)
        
        print(f"\n8. Final Order Summary:")
        print(f"  Order ID: {config['client_id']} (would be assigned by IB)")
        print(f"  Symbol: {combo_contract.symbol}")
        print(f"  Strategy: Bull Put Spread")
        print(f"  Action: {order.action}")
        print(f"  Quantity: {order.totalQuantity}")
        print(f"  Limit Price: ${order.lmtPrice:.2f}")
        print(f"  Time in Force: {order.tif}")
        print(f"  Legs:")
        for i, leg in enumerate(combo_contract.comboLegs):
            strike = sell_details['strike'] if leg.conId == sell_details['conId'] else buy_details['strike']
            print(f"    Leg {i+1}: {leg.action} {strike}P (conId: {leg.conId})")
        
        if not dry_run:
            print("\n9. Order Placement:")
            print("  WARNING: This would place a real order!")
            # In real mode, this would call: app.placeOrder(order_id, combo_contract, order)
        else:
            print("\n9. Order Placement:")
            print("  DRY RUN - Order would be placed here")
            print(f"  Call would be: app.placeOrder({config['client_id']}, combo_contract, order)")
    
    return place_order

# Run the complete test
result = test_complete_order_flow(TEST_CONFIG, dry_run=TEST_CONFIG['dry_run'])

2025-08-01 05:54:58,812 - INFO - Created option contract: UNH 20250822 240.0 P
2025-08-01 05:54:58,814 - INFO - Created option contract: UNH 20250822 230.0 P
2025-08-01 05:54:58,816 - INFO - Created combo contract for UNH with 2 legs
2025-08-01 05:54:58,817 - INFO - Created BUY limit order: Quantity=1, Price=$-1.95, TIF=DAY



=== Testing Complete Order Flow ===
DRY RUN MODE: False

1. Contract Creation:

2. Mock Contract Details (from IB):
  Sell details: {'conId': 797608409, 'symbol': 'UNH', 'strike': 240.0, 'right': 'P', 'expiry': '20250822'}
  Buy details: {'conId': 797608326, 'symbol': 'UNH', 'strike': 230.0, 'right': 'P', 'expiry': '20250822'}

3. Mock Market Data (from IB):
  240P: Bid=3.6, Ask=3.8
  230P: Bid=1.62, Ask=1.84

4. Premium Calculation:
  Calculated market premium: $197.00 using midpoint method
  Estimated premium: $197.00

5. Order Decision:
  Should place order: True

6. Combo Contract Creation:

7. Order Creation:

8. Final Order Summary:
  Order ID: 5 (would be assigned by IB)
  Symbol: UNH
  Strategy: Bull Put Spread
  Action: BUY
  Quantity: 1
  Limit Price: $-1.95
  Time in Force: DAY
  Legs:
    Leg 1: BUY 230.0P (conId: 797608326)
    Leg 2: SELL 240.0P (conId: 797608409)

9. Order Placement:


In [21]:
# Analysis of the Error 321 based on our test
print("\n" + "="*50)
print("ERROR 321 ANALYSIS")
print("="*50)

print("\nBased on our test using the exact same logic as vertical_spread_order.py:")
print("\n1. CONTRACT STRUCTURE:")
print("   - Bull Put Spread structure is CORRECT")
print("   - Leg 1: BUY 230P (lower strike) - conId: 797608326")
print("   - Leg 2: SELL 240P (higher strike) - conId: 797608409")
print("   - This creates a net credit spread as intended")

print("\n2. ORDER PARAMETERS:")
print("   - Action: BUY (correct for collecting credit)")
print("   - Limit Price: -$1.95 (negative price for credit)")
print("   - Quantity: 1")
print("   - TIF: DAY")

print("\n3. LIKELY CAUSES OF ERROR 321:")
print("   a) ACCOUNT SPECIFICATION MISSING:")
print("      - The order object doesn't specify an account")
print("      - IB Gateway may require explicit account specification")
print("      - Solution: Add order.account = 'DU9233079' (from log)")

print("\n   b) COMBO LEG ORDERING:")
print("      - Some brokers are sensitive to leg ordering in combo orders")
print("      - Current order: [BUY 230P, SELL 240P]")
print("      - Alternative: [SELL 240P, BUY 230P] (sell leg first)")

print("\n   c) EXCHANGE SPECIFICATION:")
print("      - All legs use 'SMART' exchange")
print("      - Some options may require specific exchange routing")

print("\n   d) MARKET CONDITIONS:")
print("      - Order placed during market hours (15:11:37)")
print("      - Market data was available (not -1.0 prices)")
print("      - Unlikely to be market-related")

print("\n4. RECOMMENDED FIXES:")
print("   1. Add explicit account specification to order")
print("   2. Try different leg ordering in combo contract")
print("   3. Add error handling for specific error codes")
print("   4. Consider using separate orders instead of combo")

print("\n5. TEST RESULT:")
if 'result' in locals() and result:
    print("   ✓ Order logic would proceed (market premium >= estimated)")
    print("   ✓ Contract structure is mathematically correct")
    print("   ✗ Error 321 likely due to IB Gateway validation, not logic")
else:
    print("   - Test not completed")


ERROR 321 ANALYSIS

Based on our test using the exact same logic as vertical_spread_order.py:

1. CONTRACT STRUCTURE:
   - Bull Put Spread structure is CORRECT
   - Leg 1: BUY 230P (lower strike) - conId: 797608326
   - Leg 2: SELL 240P (higher strike) - conId: 797608409
   - This creates a net credit spread as intended

2. ORDER PARAMETERS:
   - Action: BUY (correct for collecting credit)
   - Limit Price: -$1.95 (negative price for credit)
   - Quantity: 1
   - TIF: DAY

3. LIKELY CAUSES OF ERROR 321:
   a) ACCOUNT SPECIFICATION MISSING:
      - The order object doesn't specify an account
      - IB Gateway may require explicit account specification
      - Solution: Add order.account = 'DU9233079' (from log)

   b) COMBO LEG ORDERING:
      - Some brokers are sensitive to leg ordering in combo orders
      - Current order: [BUY 230P, SELL 240P]
      - Alternative: [SELL 240P, BUY 230P] (sell leg first)

   c) EXCHANGE SPECIFICATION:
      - All legs use 'SMART' exchange
      - So

In [22]:
# Optional: Test with live connection (CAUTION - set dry_run=False carefully)
def test_with_live_connection(config):
    """
    Test with actual IB Gateway connection (CAUTION!)
    Only run this if you want to test the actual connection
    """
    
    if config['dry_run']:
        print("\n=== LIVE CONNECTION TEST SKIPPED (DRY_RUN=True) ===")
        print("To test with live connection:")
        print("1. Ensure IB Gateway is running on port 4002")
        print("2. Set TEST_CONFIG['dry_run'] = False")
        print("3. Re-run this cell")
        print("\nWARNING: This will attempt to connect to IB Gateway!")
        return
    
    print("\n=== LIVE CONNECTION TEST (CAUTION!) ===")
    print("Attempting to connect to IB Gateway...")
    
    app = IBApp()
    
    try:
        # Connect to IB Gateway
        app.connect(config['ibkr_host'], config['ibkr_port'], config['client_id'])
        
        # Start the socket in a thread
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()
        
        # Wait for connection
        timeout = 10
        start_time = time.time()
        while not app.next_order_id and time.time() - start_time < timeout:
            time.sleep(0.1)
        
        if app.next_order_id:
            print(f"✓ Connected successfully! Next Order ID: {app.next_order_id}")
            
            # Test contract details request
            print("\nTesting contract details request...")
            sell_contract = app.create_option_contract(config['ticker'], config['expiry'], config['strike_sell'], "P")
            
            req_id = app.next_order_id
            app.next_order_id += 1
            
            sell_details = app.get_contract_details(sell_contract, req_id)
            
            if sell_details:
                print(f"✓ Contract details received: {sell_details}")
            else:
                print("✗ Failed to get contract details")
            
        else:
            print("✗ Failed to connect or get valid order ID")
        
    except Exception as e:
        print(f"✗ Connection error: {str(e)}")
    
    finally:
        # Always disconnect
        time.sleep(2)
        app.disconnect()
        print("Disconnected from IB Gateway")

# Run live connection test (only if dry_run=False)
test_with_live_connection(TEST_CONFIG)


=== LIVE CONNECTION TEST (CAUTION!) ===
Attempting to connect to IB Gateway...


2025-08-01 05:54:58,980 - INFO - sent startApi
2025-08-01 05:54:58,981 - INFO - REQUEST startApi {}
2025-08-01 05:54:58,982 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x005\x00\x00'
2025-08-01 05:54:58,983 - INFO - ANSWER connectAck {}
2025-08-01 05:54:59,030 - INFO - ANSWER managedAccounts {'accountsList': 'DU9233079'}
2025-08-01 05:54:59,065 - INFO - Next Valid Order ID: 1
2025-08-01 05:54:59,067 - ERROR - Error -1: 1754027698176 - 2104
2025-08-01 05:54:59,069 - ERROR - Error -1: 1754027698179 - 2107
2025-08-01 05:54:59,070 - ERROR - Error -1: 1754027698179 - 2158
2025-08-01 05:54:59,085 - INFO - Created option contract: UNH 20250822 240.0 P
2025-08-01 05:54:59,087 - INFO - REQUEST reqContractDetails {'reqId': 1, 'contract': 130893842061280: 0,UNH,OPT,20250822,,240,P,100,SMART,,USD,,,False,,,,combo:}
2025-08-01 05:54:59,090 - INFO - SENDING reqContractDetails b'\x00\x00\x0079\x008\x001\x000\x00UNH\x00OPT\x0020250822\x00240.0\x00P\x00100\x00SMART\x00\x00USD\x00\x00\x000\x00\x0

✓ Connected successfully! Next Order ID: 1

Testing contract details request...


2025-08-01 05:54:59,331 - INFO - Contract details request 1 completed


✓ Contract details received: {'conId': 797608409, 'symbol': 'UNH', 'strike': 240.0, 'right': 'P', 'expiry': '20250822'}


2025-08-01 05:55:01,296 - INFO - disconnecting
2025-08-01 05:55:01,297 - INFO - ANSWER connectionClosed {}


Disconnected from IB Gateway
